# Homework on Bass Model
## Meri Asatryan
## October 08, 2024

The **Oral-B** toothbrush brand is a great comparison for the **Samba Robotic Toothbrush**. Oral-B has been around for a while and has built up a strong following, especially with models like the iO series that introduce new technology to make brushing smarter and more effective. Your dataset, showing the number of Oral-B users in Great Britain from 2017 to 2020, highlights how they've steadily grown by keeping up with what people want.

In comparison, the **Samba Robotic Toothbrush** is a newer and more advanced product, offering a fully automated brushing experience. While Oral-B uses smart features to guide users, the Samba Robotic Toothbrush takes convenience to the next level by doing all the work for you. Oral-B's success shows how important it is to innovate, but the Samba Robotic Toothbrush could attract people who want an even more effortless experience. The data on Oral-B’s user growth can help predict how quickly the Samba Robotic Toothbrush might catch on in the market.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit

# Set seaborn style for better aesthetics
sns.set(style="whitegrid")

# 1. Loading the Dataset

def load_data(filepath):
    """
    Load the Oral-B adoption dataset.

    Parameters:
    - filepath: str, path to the CSV file.

    Returns:
    - df: pandas DataFrame with 'Year' and 'Adopters' columns.
    """
    df = pd.read_csv(filepath)
    # Ensure the data is sorted by Year
    df = df.sort_values('Year')
    return df


In [ ]:

# 2. Plotting Historical Adoption Data
def plot_historical_data(df):
    """
    Plot the historical number of adopters over time.

    Parameters:
    - df: pandas DataFrame with 'Year' and 'Adopters' columns.
    """
    plt.figure(figsize=(10,6))
    plt.plot(df['Year'], df['Adopters'], marker='o', linestyle='-', color='b')
    plt.title('Historical Adoption of Oral-B Products')
    plt.xlabel('Year')
    plt.ylabel('Number of Adopters')
    plt.grid(True)
    plt.show()

In [ ]:
# 3. Defining the Bass Diffusion Model
def bass_model(t, p, q, m):
    """
    Bass diffusion model function.

    Parameters:
    - t: Time (typically years since introduction).
    - p: Coefficient of innovation.
    - q: Coefficient of imitation.
    - m: Market potential.

    Returns:
    - Adoption at time t.
    """
    return m * ((p + q)**2 / p) * np.exp(-(p + q) * t) / ((p + q) - p * np.exp(-(p + q) * t))**2

In [ ]:
# 4. Fitting the Bass Model to the Data
def fit_bass_model(df):
    """
    Fit the Bass model to the historical adoption data.

    Parameters:
    - df: pandas DataFrame with 'Year' and 'Adopters' columns.

    Returns:
    - params: Tuple containing estimated (p, q, m).
    """
    # Preparing the data
    t = np.array(df['Year']) - df['Year'].min() + 1  
    adopters = np.array(df['Adopters'])
    cumulative_adopters = np.cumsum(adopters)

    # Initial parameter guesses
    p_initial = 0.03
    q_initial = 0.38
    m_initial = cumulative_adopters.max() * 1.5  

    initial_guess = [p_initial, q_initial, m_initial]

    # Define the cumulative Bass model
    def cumulative_bass(t, p, q, m):
        return m * (1 - np.exp(-(p + q) * t)) / (1 + (q / p) * np.exp(-(p + q) * t))

    # Fit the model
    params, covariance = curve_fit(cumulative_bass, t, cumulative_adopters, p0=initial_guess, maxfev=10000)

    return params


In [ ]:
# 5. Predicting Future Adoption
def predict_adoption(params, years_ahead, start_year):
    """
    Predict future adoption using the fitted Bass model.

    Parameters:
    - params: Tuple containing (p, q, m).
    - years_ahead: int, number of years to predict.
    - start_year: int, the last year in the historical data.

    Returns:
    - future_df: pandas DataFrame with 'Year', 'Cumulative Adopters', and 'New Adopters'.
    """
    p, q, m = params
    future_years = np.arange(start_year + 1, start_year + years_ahead + 1)
    t_future = np.arange(1, len(future_years) + 1)
    
    # Cumulative adoption for future years
    cumulative_adoption = m * (1 - np.exp(-(p + q) * t_future)) / (1 + (q / p) * np.exp(-(p + q) * t_future))
    
    # Calculate new adopters by differencing
    new_adopters = np.diff(cumulative_adoption, prepend=0)
    
    future_df = pd.DataFrame({
        'Year': future_years,
        'Cumulative Adopters': cumulative_adoption,
        'New Adopters': new_adopters
    })
    
    return future_df


In [ ]:
# 6. Plot the Fitted Bass Model
def plot_fitted_model(df, params, future_df=None):
    """
    Plot historical data and fitted Bass model.

    Parameters:
    - df: pandas DataFrame with historical 'Year' and 'Adopters'.
    - params: Tuple containing (p, q, m).
    - future_df: pandas DataFrame with future 'Year' and 'Cumulative Adopters' (optional).
    """
    p, q, m = params
    t = np.array(df['Year']) - df['Year'].min() + 1
    cumulative_adopters = np.cumsum(df['Adopters'])

    # Fitted cumulative adoption
    fitted_cumulative = m * (1 - np.exp(-(p + q) * t)) / (1 + (q / p) * np.exp(-(p + q) * t))

    plt.figure(figsize=(10,6))
    plt.plot(df['Year'], cumulative_adopters, 'o', label='Historical Cumulative Adopters')
    plt.plot(df['Year'], fitted_cumulative, '-', label='Fitted Bass Model')

    if future_df is not None:
        plt.plot(future_df['Year'], future_df['Cumulative Adopters'], '--', label='Predicted Cumulative Adopters')

    plt.title('Bass Diffusion Model Fit')
    plt.xlabel('Year')
    plt.ylabel('Cumulative Number of Adopters')
    plt.legend()
    plt.grid(True)
    plt.show()


In [ ]:
# 7. Main Execution
def main():
    # Filepath to the dataset
    data_filepath = '../data/oral_b_data.csv'

    # Load the data
    df = load_data(data_filepath)
    print("Historical Data:")
    print(df)

    # Plot historical data
    plot_historical_data(df)

    # Fit the Bass model
    params = fit_bass_model(df)
    p, q, m = params
    print(f"\nEstimated Bass Model Parameters:\n p (Innovation): {p:.4f}\n q (Imitation): {q:.4f}\n m (Market Potential): {m:.0f}")

    # Predict future adoption (e.g., next 10 years)
    years_ahead = 10
    start_year = df['Year'].max()
    future_df = predict_adoption(params, years_ahead, start_year)
    print("\nFuture Adoption Predictions:")
    print(future_df)

    # Plot fitted model with predictions
    plot_fitted_model(df, params, future_df)

    # Save predictions to CSV
    future_df.to_csv('../data/future_adoption_predictions.csv', index=False)
    print("\nFuture adoption predictions saved to 'data/future_adoption_predictions.csv'.")

if __name__ == "__main__":
    main()
